<a href="https://colab.research.google.com/github/AndaSampa/restituicao-favelas-2004/blob/main/03%20-%20CNN%20U-net%20para%20restitui%C3%A7%C3%A3o%20das%20favelas%20MDC%202004.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Restituição de favelas MDC 2004

Seguindo tutorial de referência https://towardsdatascience.com/creating-and-training-a-u-net-model-with-pytorch-for-2d-3d-semantic-segmentation-dataset-fb1f7f80fe55

## Carregando conjunto de dados de treinamento

In [1]:
import torch
import numpy as np
from skimage.io import imread
from torch.utils import data
from torch.optim import AdamW
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
cuda = torch.device("cuda")
batch_size = 75
work_folder = '/content/restituicao-favelas-2004/'

In [3]:
class Favelas2004DataSet(data.Dataset):
    def __init__(self,
                 inputs: list,
                 targets: list,
                 transform=None
                 ):
        self.inputs = inputs
        self.targets = targets
        self.transform = transform
        self.inputs_dtype = torch.float32
        self.targets_dtype = torch.long

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self,
                    index: int):
        # Select the sample
        input_ID = self.inputs[index]
        target_ID = self.targets[index]

        # Load input and target
        x, y = imread(input_ID), imread(target_ID, plugin='pil')

        # Preprocessing
        if self.transform is not None:
            x, y = self.transform(x, y)

        # Typecasting
        x, y = torch.from_numpy(x).type(self.inputs_dtype), torch.from_numpy(y).type(self.targets_dtype)

        return x, y

In [4]:
import glob

inputs = glob.glob(f'{work_folder}ortofotosMDC2004/licoes/*.JPG')
targets = glob.glob(f'{work_folder}ortofotosMDC2004/licoes/*-mask.tif')

dataset = Favelas2004DataSet(inputs=inputs,
                            targets=targets,
                            transform=None)

validation_split = .2
shuffle_dataset = True
random_seed= 42                            

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

training_dataloader = data.DataLoader(dataset=dataset,
                                    batch_size=batch_size,
                                    sampler=train_sampler)

validating_dataloader = data.DataLoader(dataset=dataset,
                                    batch_size=batch_size,
                                    sampler=valid_sampler)                                    

print(f"Quantidade de inputs, targets, treinamento e validacoes: \
    {len(inputs)}, {len(targets)}, {len(training_dataloader)}, {len(validating_dataloader)}")

Quantidade de inputs, targets, treinamento e validacoes:     7846, 7846, 84, 21


In [5]:
x, y = next(iter(training_dataloader))

In [6]:
print(x.shape, y.shape, x.dtype, y.dtype)
print(x.min(), x.max(), y.min(), y.max())

torch.Size([75, 256, 256, 3]) torch.Size([75, 256, 256]) torch.float32 torch.int64
tensor(0.) tensor(255.) tensor(0) tensor(1)


## Modelo U-Net

In [7]:
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32, pretrained=False).to("cuda")

Using cache found in /root/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


In [8]:
model

UNet(
  (encoder1): Sequential(
    (enc1conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc1norm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc1relu1): ReLU(inplace=True)
    (enc1conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc1norm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc1relu2): ReLU(inplace=True)
  )
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (encoder2): Sequential(
    (enc2conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc2norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc2relu1): ReLU(inplace=True)
    (enc2conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc2norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tra

## Treinando o modelo

In [9]:
from torch import nn

In [10]:
def dice_metric(inputs, target):
    intersection = 2.0 * (target * inputs).sum()
    union = target.sum() + inputs.sum()
    if target.sum() == 0 and inputs.sum() == 0:
        return 1.0

    return intersection / union

def dice_loss(inputs, target):
    num = target.size(0)
    inputs = inputs.reshape(num, -1)
    target = target.reshape(num, -1)
    smooth = 1.0
    intersection = (inputs * target)
    dice = (2. * intersection.sum(1) + smooth) / (inputs.sum(1) + target.sum(1) + smooth)
    dice = 1 - dice.sum() / num
    return dice

def bce_dice_loss(inputs, target):
    dicescore = dice_loss(inputs, target)
    bcescore = nn.BCELoss()
    bceloss = bcescore(inputs, target)

    return bceloss + dicescore

In [11]:
# criterion = bce_dice_loss
criterion = nn.BCELoss()

In [12]:
optimizer = AdamW(model.parameters(), 0.1)

## Fazendo inferências

In [13]:
learning_rate = 1e-3
epochs = 5


In [14]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X = torch.permute(X, (0,3,1,2)).to("cuda")
        y = y.to("cuda")
        # y = model(torch.permute(y, (0,3,1,2)))
        # Compute prediction and loss
        pred = model(X)
        print(X.size(), y.size())
        print(X.dtype, y.dtype)
        y = torch.permute(y.unsqueeze(-1), (0,3,1,2)).float()
        loss = loss_fn(pred, y).to("cuda")

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if True:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = torch.permute(X, (0,3,1,2)).to("cuda")
            y = torch.permute(y.unsqueeze(-1), (0,3,1,2))
            # y = model(torch.permute(y, (0,3,1,2)))
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [15]:
for batch, (X, y) in enumerate(training_dataloader):
    print(torch.permute(y.unsqueeze(-1), (0,3,1,2)).size())
    print(torch.permute(y.unsqueeze(-1), (0,3,1,2)))
    #model(torch.permute(i[1][1], (0,3,1,2)))
    break

torch.Size([75, 1, 256, 256])
tensor([[[[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 1, 1, 1],
          [0, 0, 0,  ..., 1, 1, 1],
          [0, 0, 0,  ..., 1, 1, 1]]],


        [[[0, 0, 0,  ..., 1, 1, 1],
          [0, 0, 0,  ..., 1, 1, 1],
          [0, 0, 0,  ..., 1, 1, 1],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]]],


        [[[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]]],


        ...,


        [[[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0]]],


        [[[0, 0

In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(training_dataloader, model, criterion, optimizer)
    test_loop(validating_dataloader, model, criterion)
print("Done!")

Epoch 1
-------------------------------
torch.Size([75, 3, 256, 256]) torch.Size([75, 256, 256])
torch.float32 torch.int64
loss: 0.695816  [    0/ 7846]
torch.Size([75, 3, 256, 256]) torch.Size([75, 256, 256])
torch.float32 torch.int64
torch.Size([75, 3, 256, 256]) torch.Size([75, 256, 256])
torch.float32 torch.int64
torch.Size([75, 3, 256, 256]) torch.Size([75, 256, 256])
torch.float32 torch.int64
torch.Size([75, 3, 256, 256]) torch.Size([75, 256, 256])
torch.float32 torch.int64
torch.Size([75, 3, 256, 256]) torch.Size([75, 256, 256])
torch.float32 torch.int64
torch.Size([75, 3, 256, 256]) torch.Size([75, 256, 256])
torch.float32 torch.int64
torch.Size([75, 3, 256, 256]) torch.Size([75, 256, 256])
torch.float32 torch.int64
torch.Size([75, 3, 256, 256]) torch.Size([75, 256, 256])
torch.float32 torch.int64
torch.Size([75, 3, 256, 256]) torch.Size([75, 256, 256])
torch.float32 torch.int64
torch.Size([75, 3, 256, 256]) torch.Size([75, 256, 256])
torch.float32 torch.int64
torch.Size([75, 3

https://github.com/qubvel/segmentation_models.pytorch#examples